In [ ]:
import import_ipynb
import requests
import sentiment
import plotly
import pandas as pd #insertdatatable
from plotly.offline import  init_notebook_mode, plot, iplot
import plotly.express as px

def callSentiment():
    file=['Bus','Grab','Ktm','Ferry', 'Flight', 'Mrt', 'Lrt', 'Taxi']
    article=['Bus','Grab','Ktm','Ferry', 'Flight', 'Mrt', 'Lrt', 'Taxi']
    before=[None]*len(file)
    after=[None]*len(file)
    nCount=[None]*len(file)
    pCount=[None]*len(file)
    nPercent=[None]*len(file)
    pPercent=[None]*len(file)
    compare=[None]*len(file)
    finalconc=[None]*len(file)
    negatives=sentiment.readNegativeWords()
    positives=sentiment.readPositiveWords()

    count=0
    for file in file:
        text=sentiment.readFile(file)
        fullwordlist = sentiment.stripNonAlphaNum(text)
        before[count]=sentiment.count(fullwordlist)
        wordlist = sentiment.removeStopwords(fullwordlist, sentiment.stopwords)
        countWords1=sentiment.count(wordlist)
        after[count]=countWords1
        nCount[count] = sentiment.calculate(negatives, wordlist)
        pCount[count] = sentiment.calculate(positives, wordlist)
        nPercent[count]= sentiment.calculatePercentage(nCount[count],after[count])
        pPercent[count] = sentiment.calculatePercentage(pCount[count],after[count])
        compare[count] = sentiment.comparePercent(nPercent[count],pPercent[count])
        finalconc[count]=sentiment.comparePositiveNegative(nCount[count],pCount[count])
        #to make graph on word frequency
        dictionary = sentiment.wordListToFreqDict(wordlist)
        sorteddict = sentiment.sortFreqDict(dictionary)
        dictionary1 = sentiment.wordListToFreqDict(fullwordlist)
        sorteddict1 = sentiment.sortFreqDict(dictionary1)
        # plot word frequencies graph for before n after removing stopwords
        plot = sentiment.plotFreq(sorteddict, file, 0)
        plot1 = sentiment.plotFreq(sorteddict1, file, 1)
        count=count+1


    df = pd.DataFrame(list(zip(article, before, after, nCount, pCount, nPercent, pPercent, compare, finalconc)),
                      columns=['article', 'before', 'after', 'negative', 'positive', '%n', '%p', 'conclusion', 'final'])

    df.to_excel(r'Sentiment.xlsx', index=False)

    print(df)

    #bar graph
    fig = px.bar(df, x = "article", y = ["negative", "positive"], title="Count positive and negative words each article")
    fig.write_html('countpositivenegative.html', auto_open=True)

    #line graph
    fig = px.scatter(df, x="article", y=['before','after'], title="Count Words Before and After Remove each article")
    fig.update_traces(mode="lines+markers+text")
    fig.write_html('countword.html', auto_open=True)